In [ ]:
# 初始化
import django_setup
import nest_asyncio
nest_asyncio.apply()

## PART-1. 进入Outline_L1 分析
<font color="orange">
事实证明，对于长文本而言，让大模型一次性提取多个层级的标题时，表现并不好，质量也不稳定。 <br>
因此，我们决定分步骤提取，先提取一级标题，再提取二级标题，最后提取三级标题。 <br>
为了接下去的测试，我们需要：<br> 
1. 项目容器： 测试项目1 <br>
2. tender_file_extraction 已经ready <br>
3. 模型输入： context, instruction, supplement, output_format, prompt_template, index_path_map, 这里都是即时生成，不做存储。<br>
4. 调用模型的函数：从task中截取和简化。 <br>
4. 标题应用函数： <br>
5. 章节的截取函数： TiptapUtils.extract_indexed_paragraphs， 在prepare_context中已调用<br>
6. 任务结果L1： 空 <br>  有了 
7. 任务结果L2： 空 <br>  有了
8. 任务结果L3： 空 <br>  有了
</font>

#### <font color="red"> 1. 准备好测试的项目容器，招标文件内容，用于存储任务结果的字段</font>

In [ ]:
# 招标文件提取内容 准备就绪
from apps.projects.models import Project, Task, TaskType
from pprint import pprint
project = Project.objects.get(project_name='测试项目1')
task = Task.objects.get(stage__project=project, type=TaskType.OUTLINE_ANALYSIS_TASK)
stage = task.stage
# --- 如果project.tender_file_extraction 为空，则需要从关联的docx_extraction_task中获取
# extraction_task = Task.objects.get(stage__project=project, type=TaskType.DOCX_EXTRACTION_TASK)
# print(extraction_task.docx_tiptap)
# print(type(extraction_task.docx_tiptap))
# pprint(extraction_task.docx_tiptap)
# project = Project.objects.get(project_name='测试项目1')
# project.tender_file_extraction = extraction_task.docx_tiptap
# project.save()
# project = Project.objects.get(project_name='测试项目1')
pprint(project.tender_file_extraction)

In [ ]:
# 存储位就绪
print(f"outline_L1存储：{project.outline_L1}")
print(f"outline_L2存储：{project.outline_L2}")
print(f"outline_L3存储：{project.outline_L3}")
print(f"outline_L1存储：{project.index_path_map_L1}")
print(f"outline_L2存储：{project.index_path_map_L2}")
print(f"outline_L3存储：{project.index_path_map_L3}")

#### <font color="red"> 2. 准备大模型的输入，并调用模型</font>

In [4]:
# 借用outline_analysis_L1.py 的函数， 进行任务准备 
# from apps.projects.services.tasks_preparation.outline_analysis_L1 import OutlineAnalysisL1
# prompt_template_L1 = OutlineAnalysisL1(project.id)._build_prompt_template()
# context_L1, index_path_map_L1 = OutlineAnalysisL1(project.id)._prepare_context(project)
# instruction_L1 = OutlineAnalysisL1(project.id)._prepare_instruction()
# supplement_L1 = OutlineAnalysisL1(project.id)._prepare_supplement()
# output_format_L1 = OutlineAnalysisL1(project.id)._prepare_output_format()
# LLM_config_L1 = OutlineAnalysisL1(project.id)._build_llm_config()

# project.index_path_map_L1 = index_path_map_L1
# project.save()

# 集成时，用以下代替，模型传参 会先存储到数据库。
from apps.projects.services.tasks_preparation.outline_analysis_L1 import OutlineAnalysisL1
params_L1=OutlineAnalysisL1(project.id).output_params()


In [ ]:
from pprint import pprint
pprint(params_L1['prompt_template'])
pprint(params_L1['context'])
pprint(params_L1['instruction'])
pprint(params_L1['supplement'])
pprint(params_L1['output_format'])
pprint(params_L1['llm_config'])
pprint(params_L1['index_path_map'])



In [ ]:
# # 模拟的prompt, 
simulated_prompt, formatted_message = OutlineAnalysisL1(project.id).simulate_prompt()
pprint(formatted_message)

#### <font color="red"> 3. 大模型的调用分析 Outline_L1</font>
<font color="orange" size="3">
- 分析获得outline_L1 <br>
- 存储结果到project.outline_L1 <br>
</font>

In [ ]:
from apps.projects.services.LLM_service.LLMcontainer import LLMService
from apps.projects.services.LLM_service._llm_data_types import LLMConfig
analyzer = LLMService(
    llm_config = LLMConfig().from_model(params_L1['llm_config']),
    context = params_L1['context'], 
    instruction = params_L1['instruction'], 
    supplement = params_L1['supplement'], 
    output_format = params_L1['output_format'], 
    prompt_template = params_L1['prompt_template'])
outline_L1 = await analyzer.process()
pprint(outline_L1)

# 将outline_L1 存储到project.outline_L1
# print(type(outline_L1), outline_L1)
# project.outline_L1 = outline_L1
# project.save()


#### <font color="red"> 4. 将Outline_L1 应用到tender_file_extraction中</font>
<font color="orange" size="3">
- 将Outline_L1 应用到tender_file_extraction <br>
- 我们需要解决：<br>
    1）outline_L1 从JSON str 转 JSON Dict？<br>
    2）如何将结果应用到tiptap JSON的 内容中？<br>
- 存储结果到project.tender_file_extraction <br>
</font>



In [ ]:
# 从数据库中获取数据
project = Project.objects.get(project_name='测试项目1')
outline_L1 = project.outline_L1
index_path_map_L1 = project.index_path_map_L1
print(f"outline_L1：{type(outline_L1)}, {outline_L1}")
print(f"index_path_map_L1：{type(index_path_map_L1)}, {index_path_map_L1}")


In [ ]:
# 将outline_L1 从JSON str 转 JSON Dict 
import json
outline_L1_dict = json.loads(outline_L1)
pprint(outline_L1_dict)
pprint(index_path_map_L1)

In [10]:
# 将outline_L1 应用到tender_file_extraction中
from apps.projects.tiptap.helpers import TiptapUtils
updated_tiptap_content = TiptapUtils.update_titles_from_list(project.tender_file_extraction, outline_L1_dict, index_path_map_L1)
project.tender_file_extraction_L1 = updated_tiptap_content
project.save()

In [ ]:
# 检查标题是否已经改到内容里去了？
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.find_all_headings(updated_tiptap_content)
pprint(headings)
print(TiptapUtils.print_headings(updated_tiptap_content))

## PART-2. 开始执行Outline_L2 分析
<font color="orange" size="3">
- 带有L1标题的tiptap JSON 内容已经ready <br>
- 我们需要解决：<br>
    1）提取章节？-> 在原文 基础上 提取段落内容（str）， 根据章节分块， 好处是,index被保留。<br>
    2）提供章节段落内容，重复类似Outline_L1的分析: <br>
        a. 准备容器，更新的文件内容，存储任务结果字段？<br>
        b. 准备大模型的输入，并调用模型？ (这个过程，需要考虑并发分析，提高效率)<br>
        c. 将结果应用到tiptap JSON的 内容中？<br>
        d. 存储结果到project.tender_file_extraction <br>
</font>

In [ ]:
# 初始化
import django_setup
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# 从数据库中获取数据
from apps.projects.models import Project, Task, TaskType
project = Project.objects.get(project_name='测试项目1')
# pprint(project.tender_file_extraction_L1)

# 提取章节
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.print_headings(project.tender_file_extraction_L1)
print(headings)

#### <font color="red"> 1. 通过outline_analysis_L2 进行传参准备</font>
<font color="orange" size="3">
- 在outline_analysis_L2 中， 根据章节分块， index被保留。<br>
- 使用了TiptapUtils.extract_chapters 工具，返回 字典对象chapter_set{}， 包含chapters 和 index_path_map 两个键名<br>
- 在outline_analysis_L2 中， 使用chapter_set['chapters'] for循环 取得chapter['paragraphs'] 作为context<br>
- 在outline_analysis_L2 的输出，将每个章节分析所需的传参 单独打包， 并返回 列表对象param_set[]<br>
</font>

In [3]:
# 通过集成的outline_anlaysis_L2 获取 大模型分析所需的 输入参数 ，并有 模拟的prompts
from apps.projects.services.tasks_preparation.outline_analysis_L2 import OutlineAnalysisL2
from pprint import pprint
params_L2_set, index_path_map_L2 = OutlineAnalysisL2(project.id).output_params()
simulated_prompt_set, formatted_prompt_set = OutlineAnalysisL2(project.id).simulate_prompt()

In [15]:
contexts = []
for params in params_L2_set:
    contexts.append(params['context'])

In [ ]:
print(len(contexts))
print(contexts[4])

In [ ]:
pprint(formatted_prompt_set[6])

#### <font color="red"> 2. 大模型逐个章节进行分析</font>

In [ ]:
from apps.projects.services.LLM_service._llm_data_types import LLMConfig
from apps.projects.services.LLM_service.LLMcontainer import LLMService
results_L2 = []
for params in params_L2_set:
    analyzer = LLMService(
        llm_config = LLMConfig().from_model(params['llm_config']),
        context = params['context'], 
        instruction = params['instruction'], 
        supplement = params['supplement'], 
        output_format = params['output_format'], 
        prompt_template = params['prompt_template']
    )
    result = await analyzer.process()
    results_L2.append(result)

In [ ]:
from apps.projects.services.LLM_service._llm_data_types import LLMConfig
from apps.projects.services.LLM_service.LLMcontainer import LLMService
results2_L2 = []
for params in params_L2_set[5:6]:
    analyzer2 = LLMService(
        llm_config = LLMConfig().from_model(params['llm_config']),
        context = params['context'], 
        instruction = params['instruction'], 
        supplement = params['supplement'], 
        output_format = params['output_format'], 
        prompt_template = params['prompt_template']
    )
    result2 = await analyzer2.process()
    results2_L2.append(result2)

In [ ]:
pprint(results2_L2[0])